In [1]:
import pyrealsense2 as rs
import numpy as np
import cv2
from ultralytics import YOLO
import time
import csv
import os
from joblib import load
from function_ObjDet  import getObject
from dimensionYolo import getObjectDimension 
#from robotiq3f_py.robotiqcontrol.GripperController import GripperController 
#from cam import cama
#from depthTracking import closing
#from depthTracking import gripTrigger

# Load Information for predicting size

In [3]:
mode = ['Basic', 'Pinch', 'Wide', 'Scissor']
loaded_rfc = load('random_forest_model76.joblib')

# Camera Set-Up

In [12]:
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)  # bgr is compatible with OpenCV
#pipeline.start(config)
pipelineProfile = pipeline.start(config)
# Align Frames to each other
align_to = rs.stream.color
align = rs.align(align_to)

(pipeline)

In [3]:
depth_sensor = pipelineProfile.get_device().first_depth_sensor()
# Set high accuracy mode
depth_sensor = pipelineProfile.get_device().first_depth_sensor()
depth_sensor.set_option(rs.option.visual_preset, rs.rs400_visual_preset.high_accuracy)
depth_scale = depth_sensor.get_depth_scale()

In [13]:
a = getObjectDimension(pipeline, depth_scale, align)

No stable object detected.


# Detect Object

In [15]:
# Get Data for model
a = getObject(pipeline)

TypeError: getObject() takes 0 positional arguments but 1 was given

# Get Mode

In [24]:
#Process Data for Model
Objdata = [[a[0][1], a[0][2], a[0][3]]]  # Change this in code of Object Detection!
dataToPredict = np.array(Objdata)
# Get prediction
predictions = int(loaded_rfc.predict(dataToPredict))

print("Vorhersage:", mode[predictions])
Objdata

Vorhersage: Basic


c:\Users\nicos\OneDrive\Desktop\Mechatronik-Projekt\Code\git\git10\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\nicos\AppData\Local\Temp\ipykernel_3168\3037824849.py:5: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predictions = int(loaded_rfc.predict(dataToPredict))


[[306.0, 209.5, 0.294]]

# Gripper

## Set-Up

## functions for simple usasge
transform data in percentage

In [52]:
def gripperPos(pos):
    gripperPos = [pos]*3 
    return gripperPos


def gripperSpeed(speed):
    gripperSpeed = [speed]*3 
    return gripperSpeed

def gripperForce(force):
    gripperForce = [force]*3 
    return gripperForce


## Connect with Gripper

In [53]:
# Initialize the GripperController object with the IP address of the server
gripper = GripperController("192.168.1.11")
gripper.activate()
print('Connected')

Gripper activateError reading data in update_status.

Connected


## Commands to gripper

### Starting position: Open Gripper in predicted Mode

#### Set mode to predicted one

In [54]:
mode = 'Pinch'
#mode = mode[predictions]
mode

'Pinch'

#### Values to open Gripper

In [55]:
# Default for Satrting position to grab object (open)
target_position = gripperPos(10)      # Hängt bei 6
force = gripperForce(40)
'''
Force ausprobieren ob das härte des Zudrückens bestimmt -> langsamere Objektdetektion in Finger
Wenn ja bei kleinen Objekten weniger force als be großen?
'''
speed = [40]
# Define if individual finger control is required
individual_control = False

gripper.command_gripper(rPRA=target_position, rSP=speed, rFR=force, rMOD=mode, rICF=individual_control)
print('command sent')
        # Wait for the gripper to reach the target positions
while ([gripper.FingerA_Position, gripper.FingerB_Position, gripper.FingerC_Position] != target_position):
        print([gripper.FingerA_Position, gripper.FingerB_Position, gripper.FingerC_Position])
        time.sleep(0.1)
            
print('position reached')

command sent
[76, 76, 76]


c:\Users\nicos\OneDrive\Desktop\Mechatronik-Projekt\Code\git\robotiq3f_py\robotiqcontrol\GripperController.py:104: UserWarning: only first value of 3d vector will be used when not using Individual Control Flag.
  warnings.warn("only first value of 3d vector will be used when not using Individual Control Flag.")


[73, 73, 73]
[68, 68, 68]
[63, 63, 63]
[58, 59, 59]
[53, 54, 54]
[49, 49, 49]
[44, 45, 45]
[40, 40, 40]
[36, 36, 36]
[31, 32, 31]
[27, 27, 27]
[22, 22, 22]
[17, 18, 17]
[12, 13, 13]
[8, 8, 8]
[8, 8, 8]
position reached


In [56]:
#print(f"target_position: {target_position}\nspeed: {speed}\nforce: {force}")

#### Send commands

In [57]:
gripper.close()

Connection closed.


### Close gripper

#### Requirement for closing the gripper

In [58]:
'''
Nicht aus Kamera ausbrechen, sondern drin bleiben und ditanz überpfüfen

Wenn kleiner als bestimmter Wert greifer Schlißen
'''
closeGripper = False

In [27]:
#pipelineProfile = pipeline.start(config)
device = pipelineProfile.get_device()
depthSensor = device.first_depth_sensor()
if depthSensor:
    depthSensor.set_option(rs.option.visual_preset, rs.rs400_visual_preset.high_accuracy)
    print('High Accuracy activated')

High Accuracy activated


In [ ]:
# b = 

#### Values to close Gripper

In [37]:
#pipeline.stop()
#pipeline.start(config)
closeGripper = closing(pipeline)


Starting closing function...
Waiting for frames...
Predicting objects...
Predicting objects...
Predicting objects...
Predicting objects...
Predicting objects...
Predicting objects...
Predicting objects...
Predicting objects...
Predicting objects...
Predicting objects...
Predicting objects...
Object 'cell phone' detected closer than 0.2m. Aborting...
Closing function finished.


In [60]:
closeGripper = True

#### Send commands

In [62]:
if closeGripper:
        target_position = [255]*3 # Closing position
        
        
        movement = [300]*3
        gripper.command_gripper(rPRA=target_position, rSP=speed, rFR=force, rMOD=mode, rICF=individual_control)
        print('command sent')
        movement.append([gripper.FingerA_Position, gripper.FingerB_Position, gripper.FingerC_Position])
        i = 1
        
        
        
        gripper.command_gripper(rPRA=target_position, rSP=speed, rFR=force, rMOD=mode, rICF=individual_control)

                # Wait for the gripper to reach the target positions
        while ([gripper.FingerA_Position, gripper.FingerB_Position, gripper.FingerC_Position] != target_position):
                
                fingerPos = [gripper.FingerA_Position, gripper.FingerB_Position, gripper.FingerC_Position]
                movement[i] = fingerPos
                print('checking')
                if fingerPos == movement[i-1]:
                        movement.append(fingerPos)
                if len(movement):
                        break
                        
                else: 
                        movement[0] = [300]*0
                print([gripper.FingerA_Position, gripper.FingerB_Position, gripper.FingerC_Position])
                print('Sleep')
                time.sleep(0.1)
                #time.sleep(0.1)

command sent
checking


# Wait for next instruction

In [36]:
print("Ready for next Step")

Ready for next Step


# Open Gripper to give Object

In [37]:
'''
Mit mediapipe hand landmarks checken und wenn an Gripper dann loslassen 
Oder wenn Widerstand gespürt wird?
'''

'\nMit mediapipe hand landmarks checken und wenn an Gripper dann loslassen \nOder wenn Widerstand gespürt wird?\n'

In [ ]:
target_position = [0]*3 # Closing position
movement = [300]*3
gripper.command_gripper(rPRA=target_position, rSP=speed, rFR=force, rMOD=mode, rICF=individual_control)
print('command sent')
movement.append([gripper.FingerA_Position, gripper.FingerB_Position, gripper.FingerC_Position])
i = 1

        # Wait for the gripper to reach the target positions
while ([gripper.FingerA_Position, gripper.FingerB_Position, gripper.FingerC_Position] != target_position):
       # while any([gripper.FingerA_Current, gripper.FingerB_Current, gripper.FingerC_Current]) !=0: #probleme mit zugreifen, da bei objekterkennung automatisch gestoppt wird?
        # while()
        # If no futher movemnt because of object in hand -> stop moving!
        fingerPos = [gripper.FingerA_Position, gripper.FingerB_Position, gripper.FingerC_Position]
        movement[i] = fingerPos
        print('checking')
        if fingerPos == movement[i-1]:
                movement.append(fingerPos)
        if len(movement):
                break
                
        else: 
                movement[0] = [300]*0
        print([gripper.FingerA_Position, gripper.FingerB_Position, gripper.FingerC_Position])
        print('Sleep')
        time.sleep(0.1)
print('end')

command sent
checking
end


In [39]:
len(movement)

46

# Stop


In [6]:
pipeline.stop() 
gripper.close()

RuntimeError: stop() cannot be called before start()